# Read and manipulate some data in .pvt files

In [22]:
# Here we can implement some criteria in order to modify PVT values
def calculate_new_property_value(P_value, T_value, previous_property_value):
    '''
    Here we can add some criteria in order to change values of
    a variable selected in "property_index"
    '''
    if previous_property_value > 100.0:
        new_property_value = 100.0
        return f"{new_property_value:.6e}"
    else:
        return f"{previous_property_value:.6e}"

In [23]:
import pathlib

# INPUT:
TABLE_NAME = "Tabela_Tampliada_200"

# Filepath
filename = pathlib.Path("W:\\" + TABLE_NAME + ".tab")

# Read original table
original_lines = [i.strip().split() for i in open(filename).readlines()]

converted_lines = original_lines.copy()

names = converted_lines[21][2][1:-1].split(',')

# Variable name
property_index = names.index("ROG")

pvt_points = [l for l in converted_lines[23:] if 'PVTTABLE' in l]
converted_pvt_points = pvt_points.copy()

for i, (_, _, _, point) in enumerate(pvt_points):
    line_values = point[1:-1].split(',')
    
    def get_value(index):
        return float(line_values[index])
    
    P_value = get_value(names.index("PT"))
    T_value = get_value(names.index("TM"))

    property_value = get_value(property_index)
    line_values[property_index] = calculate_new_property_value(P_value, T_value, property_value)
    
    converted_point = "(" + ",".join(line_values) + ")"
    converted_pvt_points[i][3] = converted_point

converted_lines[23:] = converted_pvt_points

out_filename = pathlib.Path("W:\\" + TABLE_NAME + "_modified.tab")
converte_file = open(out_filename, "w")
with converte_file as f:
    for _list in converted_lines:
        f.write(" ".join(_list) + '\n')